# EDA totale

Importiamo i pacchetti

In [1]:
import numpy as np

import pandas as pd
import geopandas as gp
from shapely.geometry import Polygon
import json

import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import plotly.graph_objects as go

definiamoci il path ai dati per il mio computer

In [2]:
data_path = "C:/Users/ADMIN/Desktop/Data Science/Lavandini/data/"
file_path_save = data_path + "save_data/"
file_path_proc = "C:/Users/ADMIN/Desktop/Data Science/Lavandini/data/processed/"
file_path_raw = "C:/Users/ADMIN/Desktop/Data Science/Lavandini/data/data_trentino/raw/"
file_path_ext = "C:/Users/ADMIN/Desktop/Data Science/Lavandini/data/data_trentino/external/"

---
## Dati Meteo
Andiamo ora a visualizzare i dati meteo e organizziamoli in un dataframe che saranno poi utili in fase ti classificazione e regressione.

### Precipitazioni
Per prima cosa andiamo a creare e visualizzare il dataframe con le precipitazioni:  

In [3]:
# Importiamo il file csv
tags = ["DATE", "SQUAREID", "RAIN"]
precip_df = pd.read_csv(file_path_raw + 'precipitation-trentino.csv', names=tags)
del(tags)


Aggiustiamo la colonna **DATE** similmente a come già fatto in precedenza per `set_df`

In [4]:
precip_df["DATE"] = pd.to_datetime(precip_df["DATE"], format="%Y%m%d%H%M")
precip_df["MONTH"] = pd.to_datetime(precip_df["DATE"]).dt.month
precip_df["DAY"] = pd.to_datetime(precip_df["DATE"]).dt.day
precip_df["MINUTE"] = pd.to_datetime(precip_df["DATE"]).dt.hour * 60 + pd.to_datetime(precip_df["DATE"]).dt.minute
precip_df["WEEK_DAY"] = pd.to_datetime(precip_df["DATE"]).dt.day_of_week

# ordiniamoli in base alla data
precip_df = precip_df.sort_values(by='DATE')
precip_df = precip_df.reset_index()
del precip_df["index"]

precip_df

,DATE,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY
0,2013-11-01 00:00:00,2383,1,11,1,0,4
1,2013-11-01 00:00:00,5429,2,11,1,0,4
2,2013-11-01 00:00:00,5205,1,11,1,0,4
3,2013-11-01 00:00:00,6754,1,11,1,0,4
4,2013-11-01 00:00:00,7436,1,11,1,0,4
...,...,...,...,...,...,...,...
3489412,2013-12-26 21:10:00,5193,1,12,26,1270,3
3489413,2013-12-26 21:10:00,4967,1,12,26,1270,3
3489414,2013-12-26 21:10:00,4612,1,12,26,1270,3
3489415,2013-12-26 21:10:00,9697,1,12,26,1270,3


In [5]:
i = 2307596
precip_df[i:i+4]

,DATE,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY
2307596,2013-11-30 01:40:00,9582,1,11,30,100,5
2307597,2013-11-30 01:40:00,7225,1,11,30,100,5
2307598,2013-12-02 00:50:00,9033,1,12,2,50,0
2307599,2013-12-02 00:50:00,9866,1,12,2,50,0


Si può vedere che per alcuni orari mancano dei dati, questo è perché...

Aggiungiamo anche il file `precipitation-trentino-data-availability.csv` così da avere dei valori `true` e `false` se i dati sono presenti o mancanti.

In [6]:
# Importiamo il file csv
tags = ["DATE", "IS_RAIN"]
precip_avail_df = pd.read_csv(file_path_raw + 'precipitation-trentino-data-availability.csv', names=tags)
del(tags)

# Aggiustiamo la data
precip_avail_df["DATE"] = pd.to_datetime(precip_avail_df["DATE"], format="%Y%m%d%H%M")

precip_avail_df

,DATE,IS_RAIN
0,2013-11-01 00:00:00,True
1,2013-11-01 00:10:00,True
2,2013-11-01 00:20:00,True
3,2013-11-01 00:30:00,True
4,2013-11-01 00:40:00,True
...,...,...
8779,2013-12-31 23:10:00,False
8780,2013-12-31 23:20:00,False
8781,2013-12-31 23:30:00,False
8782,2013-12-31 23:40:00,False


Ora uniamo i due dataframe così da avere un quadro completo  
Inoltre gestiamo i NaN nelle colonne SQUAREID e RAIN, sostituiamoli con degli zeri

In [7]:
precip_df = pd.merge(precip_df, precip_avail_df,on="DATE", how="right")

precip_df["MONTH"] = pd.to_datetime(precip_df["DATE"]).dt.month
precip_df["DAY"] = pd.to_datetime(precip_df["DATE"]).dt.day
precip_df["MINUTE"] = pd.to_datetime(precip_df["DATE"]).dt.hour * 60 + pd.to_datetime(precip_df["DATE"]).dt.minute
precip_df["WEEK_DAY"] = pd.to_datetime(precip_df["DATE"]).dt.day_of_week

# ordiniamoli in base alla data
precip_df = precip_df.sort_values(by='DATE')
precip_df = precip_df.reset_index()
del precip_df["index"]

# gestiamo i NaN nelle colonne SQUAREID e RAIN, sostituiamoli con uno 0
precip_df['SQUAREID'] = precip_df['SQUAREID'].fillna(0).astype(np.int64)
precip_df['RAIN'] = precip_df['RAIN'].fillna(0).astype(np.int64)
del precip_df["DATE"]

precip_df

,DATE,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN
0,2013-11-01 00:00:00,2383,1,11,1,0,4,True
1,2013-11-01 00:00:00,7930,1,11,1,0,4,True
2,2013-11-01 00:00:00,7559,1,11,1,0,4,True
3,2013-11-01 00:00:00,9815,2,11,1,0,4,True
4,2013-11-01 00:00:00,6140,1,11,1,0,4,True
...,...,...,...,...,...,...,...,...
3490952,2013-12-31 23:10:00,0,0,12,31,1390,1,False
3490953,2013-12-31 23:20:00,0,0,12,31,1400,1,False
3490954,2013-12-31 23:30:00,0,0,12,31,1410,1,False
3490955,2013-12-31 23:40:00,0,0,12,31,1420,1,False


---

# Creiamo un dataframe totate

Carichiamo i dataframe `power_time_nov_df` e `power_time_dec_df` dalla cartella _save_data_:


In [8]:
power_time_nov_df = pd.read_csv(file_path_save + "power_time_nov_df.csv")
power_time_dec_df = pd.read_csv(file_path_save + "power_time_dec_df.csv")
power_time_dec_df

,LINESET,POWER,DAY,MINUTE,WEEK_DAY,SQUAREID,NR_UBI,TOT_UBI,W_POWER,MONTH
0,DG1000420,31726.077408,1,0,6,4037,2,2,31726.077408,12
1,DG1000420,31726.077408,1,0,6,4154,13,13,31726.077408,12
2,DG1000420,31726.077408,1,0,6,4155,20,20,31726.077408,12
3,DG1000420,31726.077408,1,0,6,4156,2,2,31726.077408,12
4,DG1000420,31726.077408,1,0,6,4269,1,1,31726.077408,12
...,...,...,...,...,...,...,...,...,...,...
11298379,DG1056626,65469.601728,31,1430,1,8502,1,1,65469.601728,12
11298380,DG1056626,65469.601728,31,1430,1,8503,4,4,65469.601728,12
11298381,DG1056626,65469.601728,31,1430,1,8621,1,3,21823.200576,12
11298382,DG1056626,65469.601728,31,1430,1,8972,25,25,65469.601728,12


Uniamo i due dataframe e facciamo un merge con `precip_df`

In [20]:
temp_df = pd.concat([power_time_nov_df, power_time_dec_df], ignore_index=True)

final_df = pd.merge(precip_df, temp_df, on=["SQUAREID", "MONTH", "DAY", "MINUTE", "WEEK_DAY"], how="right")
del temp_df

final_df

,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN,LINESET,POWER,NR_UBI,TOT_UBI,W_POWER
0,4037,NaN,11,1,0,4,NaN,DG1000420,32348.159136,2,2,32348.159136
1,4154,NaN,11,1,0,4,NaN,DG1000420,32348.159136,13,13,32348.159136
2,4155,NaN,11,1,0,4,NaN,DG1000420,32348.159136,20,20,32348.159136
3,4156,NaN,11,1,0,4,NaN,DG1000420,32348.159136,2,2,32348.159136
4,4269,NaN,11,1,0,4,NaN,DG1000420,32348.159136,1,1,32348.159136
...,...,...,...,...,...,...,...,...,...,...,...,...
22227259,8502,NaN,12,31,1430,1,NaN,DG1056626,65469.601728,1,1,65469.601728
22227260,8503,NaN,12,31,1430,1,NaN,DG1056626,65469.601728,4,4,65469.601728
22227261,8621,NaN,12,31,1430,1,NaN,DG1056626,65469.601728,1,3,21823.200576
22227262,8972,NaN,12,31,1430,1,NaN,DG1056626,65469.601728,25,25,65469.601728


In [25]:
final_df[final_df["RAIN"]==np.NaN][50:100]

,SQUAREID,RAIN,MONTH,DAY,MINUTE,WEEK_DAY,IS_RAIN,LINESET,POWER,NR_UBI,TOT_UBI,W_POWER


Come da consegna consideriamo solo i giorni feriali:

In [ ]:
final_df = final_df[final_df["WEEK_DAY"] == []]